In [8]:
library(slurmR)
library(Seurat)
library(pbapply)
library(EnvStats)
library(tidyverse)
library(SeuratData)

pbo <- pboptions(type='timer')

Loading required package: parallel

slurmR default option for `tmp_path` (used to store auxiliar files) set to:
  /mnt/scratchb/fmlab/baker02/crigen/notebooks
You can change this and checkout other slurmR options using: ?opts_slurmR, or you could just type "opts_slurmR" on the terminal.



In [2]:
eccite <- LoadData(ds = "thp1.eccite")
DefaultAssay(object = eccite) <- 'RNA'

Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning me

In [3]:
v2_rna_fraction <- 0.14
df <- eccite[[]] %>%
                as.data.frame %>% 
                mutate(rna_pop_estimate = round(nCount_RNA / v2_rna_fraction))

counts <- eccite[['RNA']]@data

In [20]:
df_to_list <- function(df) {
    row_list <- split(df, seq(nrow(df)))
    return (row_list)
}

estimate_mRNA <- function(cell, counts) {
   rna_sample <- cell$nFeature_RNA
   rna_pop <- cell$rna_pop_estimate
    
    cell_counts <- counts[, rownames(cell)]
    cell_counts <- cell_counts[cell_counts != 0]
    
    count_estimates <- lapply(cell_counts, function(gene) {
        gene_count <- cell_counts[gene]
        
        if (gene_count == 0) {
            mRNA_estimate <- 0
        } else {
            mRNA_estimate <- ehyper(gene_count, total=rna_pop, k=rna_sample)$parameter[1]
            mRNA_estimate <- as.integer(mRNA_estimate)
        }
        
        return (mRNA_estimate)
    })
    
    return (count_estimates)
}

In [21]:
ctrl <- df %>% filter(gene == 'NT' & replicate == 'rep1') %>% df_to_list()

In [22]:
njobs <- 500

if (njobs > length(ctrl)) {
    njobs <- length(ctrl)
}

In [26]:
estimates <- Slurm_lapply(ctrl, estimate_mRNA, counts=counts, njobs=njobs, mc.cores=1, sbatch_opt=list(mem='4G', time='1-00:00:00'))

The path /mnt/scratchb/fmlab/baker02/crigen/notebooks/slurmr-job-4e6c4cc1558f already exists. Since `overwrite = TRUE`,slurmR will remove the previous data.

Submitting job...


Submitted batch job 14780495


In [27]:
count_estimates <- counts

In [39]:
pblapply(1:length(estimates), function(cell_i) {
    cell_estimates <- estimates[[cell_i]]
    
    lapply(cell_estimates, function(gene) {
        count_estimates[gene, cell_i] <- cell_estimates[[gene]]
    })
})

  |                                                  | 0 % ~calculating  


In [ ]:
# count_estimates[names(estimates), 1]
count_estimates[names(estimates[[1]]), 1]